# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

PATH = "UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [2]:
tree = trees[0]
print(tree)

TokenList<У, домі, римського, патриція, Руфіна, була, прегарна, фреска, ,, зображення, Венери, та, Адоніса, .>


In [3]:
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [4]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [5]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [6]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

In [7]:
def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'В

In [8]:
#  (6, 12), (7, 12) AND (10, 13)
trace_actions(trees[9])

Stack: ['ROOT_0']
Queue: ['Лесю_1', 'Українку_2', 'цензурували_3', 'надзвичайно_4', 'пильно_5', ',_6', 'адже_7', 'в_8', 'радянському_9', 'іконостасі_10', 'вона_11', 'мала_12', 'постати_13', 'бездоганною_14', 'марксисткою_15', ',_16', '«_17', 'пролетарською_18', 'інтернаціоналісткою_19', '»_20', ',_21', '«_22', 'другом_23', 'робітників_24', '»_25', ',_26', 'предтечею_27', 'соціалістичного_28', 'реалізму_29', 'et_30', 'cetera_31', '._32']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'Лесю_1']
Queue: ['Українку_2', 'цензурували_3', 'надзвичайно_4', 'пильно_5', ',_6', 'адже_7', 'в_8', 'радянському_9', 'іконостасі_10', 'вона_11', 'мала_12', 'постати_13', 'бездоганною_14', 'марксисткою_15', ',_16', '«_17', 'пролетарською_18', 'інтернаціоналісткою_19', '»_20', ',_21', '«_22', 'другом_23', 'робітників_24', '»_25', ',_26', 'предтечею_27', 'соціалістичного_28', 'реалізму_29', 'et_30', 'cetera_31', '._32']
Relations: []
Actions.RIGHT
Stack: ['ROOT_0', 'Лесю_1', 'Українку_2']
Queue: ['цензурували

Stack: ['ROOT_0', 'цензурували_3', 'пильно_5']
Queue: []
Relations: [(2, 1), (1, 3), (3, 0), (4, 5), (5, 3), (9, 10), (8, 10), (11, 12), (13, 12), (14, 15), (15, 13), (18, 19), (17, 19), (16, 19), (19, 15), (20, 19), (22, 23), (21, 23), (23, 15), (24, 23), (25, 23), (26, 27), (27, 15), (28, 29), (29, 27), (30, 15), (31, 30)]
Actions.REDUCE
Stack: ['ROOT_0', 'цензурували_3']
Queue: []
Relations: [(2, 1), (1, 3), (3, 0), (4, 5), (5, 3), (9, 10), (8, 10), (11, 12), (13, 12), (14, 15), (15, 13), (18, 19), (17, 19), (16, 19), (19, 15), (20, 19), (22, 23), (21, 23), (23, 15), (24, 23), (25, 23), (26, 27), (27, 15), (28, 29), (29, 27), (30, 15), (31, 30)]
Actions.REDUCE
Stack: ['ROOT_0']
Queue: []
Relations: [(2, 1), (1, 3), (3, 0), (4, 5), (5, 3), (9, 10), (8, 10), (11, 12), (13, 12), (14, 15), (15, 13), (18, 19), (17, 19), (16, 19), (19, 15), (20, 19), (22, 23), (21, 23), (23, 15), (24, 23), (25, 23), (26, 27), (27, 15), (28, 29), (29, 27), (30, 15), (31, 30)]
Actions.REDUCE
Gold relations:

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [9]:
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

## Prepare train and test data

In [10]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [11]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)

Number of words: 14
Number of actions: 29
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q1-word': 'домі', 'q1-tag': 'NOUN', 'q2-tag': 'ADJ', 'q3-tag': 'NOUN'}
{'s0-word': 'У', 's0-lemma': 'у', 's0-tag': 'ADP', 's1-tag': 'ROOT', 'q0-word': 'домі', 'q0-lemma': 'дім', 'q0-tag': 'NOUN', 'q1-word': 'римського', 'q1-tag': 'ADJ', 'q2-tag': 'NOUN', 'q3-tag': 'PROPN'}
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'домі', 'q0-lemma': 'дім', 'q0-tag': 'NOUN', 'q1-word': 'римського', 'q1-tag': 'ADJ', 'q2-tag': 'NOUN', 'q3-tag': 'PROPN'}
{'s0-word': 'домі', 's0-lemma': 'дім', 's0-tag': 'NOUN', 's1-tag': 'ROOT', 'q0-word':

In [12]:
train_features, train_labels = [], []
for tree in trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [13]:
# Test data

with open(PATH + "/uk_iu-ud-dev.conllu", "r") as f:
    data = f.read()
test_trees = parse(data)

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

25820 25820


## Train a classifier

In [14]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [15]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  111126


In [16]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

190298 25820


In [17]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 546 epochs took 202 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='saga',
          tol=0.0001, verbose=1, warm_start=False)

In [18]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.86      0.87      0.86      6371
      reduce       0.85      0.78      0.81      6875
       right       0.75      0.79      0.77      5996
       shift       0.85      0.87      0.86      6578

   micro avg       0.83      0.83      0.83     25820
   macro avg       0.83      0.83      0.83     25820
weighted avg       0.83      0.83      0.83     25820



## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [19]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if log:
                print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
                print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
                print("Relations:", relations)
                print(action)
                print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [20]:
print([node["form"] for node in test_trees[0]])
print(dep_parse(test_trees[0], lrc, vec))
print([(node["id"], node["head"]) for node in test_trees[0]])

['Дідусь', ',', 'той', 'що', 'атестував', ',', 'посміхнувся', 'й', 'спитав', ':']
Stack: ['ROOT_0']
Queue: ['Дідусь_1', ',_2', 'той_3', 'що_4', 'атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: []
right
Stack: ['ROOT_0', 'Дідусь_1']
Queue: [',_2', 'той_3', 'що_4', 'атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: [(1, 0)]
right
Stack: ['ROOT_0', 'Дідусь_1', ',_2']
Queue: ['той_3', 'що_4', 'атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: [(1, 0), (2, 1)]
shift
Stack: ['ROOT_0', 'Дідусь_1', ',_2', 'той_3']
Queue: ['що_4', 'атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: [(1, 0), (2, 1)]
shift
Stack: ['ROOT_0', 'Дідусь_1', ',_2', 'той_3', 'що_4']
Queue: ['атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: [(1, 0), (2, 1)]
left
Stack: ['ROOT_0', 'Дідусь_1', ',_2', 'той_3']
Queue: ['атестував_5', ',_6', 'посміхнувся_7', 'й_8', 'спитав_9', ':_10']
Relations: 

In [21]:
print([node["form"] for node in test_trees[3]])
print(dep_parse(test_trees[3], lrc, vec))
print([(node["id"], node["head"]) for node in test_trees[3]])

['—', 'Видно', '…']
Stack: ['ROOT_0']
Queue: ['—_1', 'Видно_2', '…_3']
Relations: []
shift
Stack: ['ROOT_0', '—_1']
Queue: ['Видно_2', '…_3']
Relations: []
left
Stack: ['ROOT_0']
Queue: ['Видно_2', '…_3']
Relations: [(1, 2)]
right
Stack: ['ROOT_0', 'Видно_2']
Queue: ['…_3']
Relations: [(1, 2), (2, 0)]
right
[(1, 2), (2, 0), (3, 2)]
[(1, 2), (2, 0), (3, 2)]


In [ ]:
total, tp = 0, 0
for tree in test_trees:
    tree = [t for t in tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec, log=False)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))

## Find non-projective trees

In [ ]:
def is_non_projective(tree):
    relations = [[i['id'], i['head']] for i in tree if type(i["id"])==int]
    for rel in relations:
        for ref_rel in relations:
            a, c = sorted(rel)
            b, d = sorted(ref_rel)
            if a < b and b < c and c < d:
                return True
    return False

total_non_pr = 0
tree_ids = []
for i in range(len(trees)):
    if is_non_projective(trees[i]):
        total_non_pr += 1
        tree_ids.append(i)

# 8% (e.g., tree no. 28)
print("The number of non-projective trees is {} ({} out of {}).".
      format(round(total_non_pr * 100 / len(trees), 2), total_non_pr, len(trees)))